In [6]:
import openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import tiktoken
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain.schema import StrOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import langchain_community.embeddings.huggingface
# help(langchain_community.embeddings.huggingface)
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory

import os, dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# embeddings = OpenAIEmbeddings(deployment="textembedding", chunk_size = 16, api_key = os.environ["OPENAI_API_KEY"])
# index_name = "SCLC"

model_name = "bge-large-en-v1.5"
model_kwargs = {"device": "cpu"}
# model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    # model_name=model_name, 
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs)

index_name = "indexes/ChestPainRubrics"

# store = FAISS.load_local(index_name, embeddings)
import db_firestore as db
store = db.get_store(index_name, embeddings=embeddings)

In [42]:
TEMPLATE = """You are a teacher for medical students. Your task is to provide an overall assessment of a student's diagnosis, based on the 
You will be provided with the following information:
1. The rubrics that the student should be judged based upon.
2. The conversation history between the medical student and the patient.
3. The final diagnosis that the student will make.

=================================================================

Your task is as follows:
1. Your grading should touch on every part of the rubrics, and grade the student holistically.
Finally, provide an overall grade for the student.

Some additional information that is useful to understand the rubrics:
- The rubrics are segmented, with each area separated by dashes, such as "----------" 
- There will be multiple segments on History Taking. For each segment, the rubrics and corresponding grades will be provided below the required history taking.
- For History Taking, you are to grade the student based on the rubrics, by checking the chat history between the patients and the medical student.
- There is an additional segment on Presentation, differentials, and diagnosis. The 


=================================================================


Here are the rubrics for grading the student:
<rubrics>

{context}

</rubrics>

=================================================================
You are to give a comprehensive judgement based on the student's diagnosis, with reference to the above rubrics.

Here is the chat history between the medical student and the patient:

<history>

{history}

</history>
=================================================================


Student's final diagnosis:
<diagnosis>
    {question}
</diagnosis>

=================================================================

Your grade:
"""

prompt = PromptTemplate(
    input_variables = ["question", "context", "history"],
    template = TEMPLATE
)
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k":2})
def format_docs(docs):
    return "\n--------------------\n".join(doc.page_content for doc in docs)


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)



In [43]:

# llm_gpt4 = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
llm_gpt4 = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)

## some code to generate the fake history easier

In [10]:
fake_history = []
i = " "

### Send no message to end it
while i:
    i = input()
    if i:
        fake_history.append(i)

print(fake_history)

[]


In [11]:
## Sample history:

# fake_history = [
#     "student: How are you mr tan?",
#     "patient: I am not feeling well...",
#     "student: oh, that's terrible! tell me more about what happened to you",
#     "patient: well I got this chest pain",
#     "student: I'm sorry to hear that. How long has your chest pain been?",
#     "patient: I can't remember",
# ]

fake_history = """
student: How are you mr tan?
patient: I am not feeling well...
student: oh, that's terrible! tell me more about what happened to you
patient: well I got this chest pain
student: I'm sorry to hear that. How long has your chest pain been?
patient: I can't remember
""".strip().split("\n")

for i in fake_history: print(i)

student: How are you mr tan?
patient: I am not feeling well...
student: oh, that's terrible! tell me more about what happened to you
patient: well I got this chest pain
student: I'm sorry to hear that. How long has your chest pain been?
patient: I can't remember


In [44]:
fake_history = """student: Are you okay?
patient: No, I'm not okay.
student: What happened to you
patient: I suddenly had very bad chest pain that came suddenly.
student: Oh no, that's terrible! tell me more about your chest pain?
patient: I suddenly had very bad chest pain that came suddenly. It's a tearing pain and it radiates to my back. It started 1-2 hours ago this morning and hasn't improved.
student: how bad is the pain, on a scale of 1 to 5?
patient: The pain is a 4.
student: did you have any history of chest pain in the past?
patient: Yes, I had severe pain before due to cholangitis.
student: how does it compare to that?
patient: It's the same.
student: how does it compare to that? is the pain worse?
patient: Yes, the pain is worse.
student: have you ever had any headaches before this?
patient: No, I haven't had any headaches before this.
student: how about fainting?
patient: I fainted once.
student: can you tell me more about that time you fainted?
patient: I fainted once, but I didn't hit my head or anything.
student: okay great... how about nausea? do you feel like vomiting?
patient: No, I don't feel like vomiting.
student: any blood in the poop?
patient: No blood in poop.
student: alright that's great! can I know if you have any pains anywhere else?
patient: No, I don't have any other pains.
student: So only head pain?
patient: No, chest pain too.
student: ok. let me take a look at your skin. do you have rashes?
patient: No, I don't have rashes.
student: did you get the flu recently? any fever or cough so far?
patient: No, I didn't get the flu recently. No fever or cough.
student: good. Can I know a bit more about your health history?
patient: I have high blood pressure, high cholesterol, and type 2 diabetes. I also had gallstones and biliary colic. I was recently hospitalized for acute cholangitis and had a procedure to remove the stones. My father had a heart attack, and my mother had a collagen tissue disorder.
student: that's very detailed, thank you! do you have any allergies?
patient: No, I don't have any allergies.
student: great. what medication are you taking now for the high blood pressure and cholestrol?
patient: I take Enalapril for high blood pressure and Rosuvastatin for cholesterol.
student: do you take them regularly?
patient: No, I'm not good at taking them regularly.
student: can you tell me what are the doses, and when do you take them?
patient: I don't know the doses or when to take them.
student: that's not good! you should take them at regular intervals okay?
patient: Okay, I will take them regularly.
student: other than these medication, do you take anything else?
patient: No, I don't take anything else.
student: for diabetes?
patient: Yes, I have diabetes.
student: what medication do you take for diabetes?
patient: Metformin.
student: do you smoke?
patient: Yes, I smoke.
student: ok, how many packets do you usually smoke a day?
patient: I smoke 10 sticks a day.
student: smoking for how long already?
patient: 30 years.
student: any sexual activity? STDs?
patient: No recent sexual activity. No STDs.
student: ok. how about your parents? do they have history of chest pains as well
patient: Yes, both parents have chest pains.
student: ok, I will write you some medication for your pain. You wait here first okay?
patient: Okay, thank you."""

In [45]:
## RESET MEMORY
## To reset the memory (if it screws up), rerun this cell
memory = ConversationSummaryBufferMemory(llm=llm, memory_key="chat_history", input_key="question" )

def x(_): 
    return fake_history
chain = (
    {
        "context": retriever | format_docs, 
        "history": x,
        "question": RunnablePassthrough(),
        } | 

    # prompt # | 
    LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True) #|
     
    # StrOutputParser()
)

In [46]:
diagnosis = """
From the sudden chest pain, it is likely it's aortic dissection. The radiation and intensity signifies it. 
Also possible to be acute pancreatitis, since there's back pain, and history of problems.
Doubtful it's oesophageal rupture, since there was no vomiting or nausea. 
"""

results = chain.invoke(diagnosis)
print(results.get("text"))



> Entering new LLMChain chain...
Prompt after formatting:
You are a teacher for medical students. Your task is to provide an overall assessment of a student's diagnosis, based on the 
You will be provided with the following information:
1. The rubrics that the student should be judged based upon.
2. The conversation history between the medical student and the patient.
3. The final diagnosis that the student will make.


Your task is as follows:
1. Your grading should touch on every part of the rubrics, and grade the student holistically.
Finally, provide an overall grade for the student.

Some additional information that is useful to understand the rubrics:
- The rubrics are segmented, with each area separated by dashes, such as "----------" 
- There will be multiple segments on History Taking. For each segment, the rubrics and corresponding grades will be provided below the required history taking.
- For History Taking, you are to grade the student based on the rubrics, by checking 

In [48]:
# print(results.text)